Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [246]:
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappush, heappop
from collections import namedtuple
from typing import List
import heapq
import heapq
from typing import List, Tuple, Callable
import numpy as np

In [247]:
# Define the Action type
Action = namedtuple('Action', ['pos1', 'pos2'])

# Define puzzle dimension
PUZZLE_DIM = 6

In [248]:
def available_actions(state: np.ndarray) -> List[Action]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions

"""Apply the action to the state."""
def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [249]:
"""Calculate the distance of Manhattan between the current state and the final state."""
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    distance = 0
    for num in range(1, PUZZLE_DIM**2):  # ignore 0 (the empty space)
        x1, y1 = np.where(state == num)
        x2, y2 = np.where(goal == num)
        distance += abs(x1[0] - x2[0]) + abs(y1[0] - y2[0])
    return distance


__Reconstruct the path__

*Parameters*:
- cameFrom (dict): Map of state to its predecessor state.
- current (Tuple[int]): Current state (goal state tuple).
- shape (Tuple[int]): Shape of the original puzzle.

*Returns*:
- List[np.ndarray]: Reconstructed path from start to goal.

In [250]:
'''Reconstruct the path from start to goal using the cameFrom dictionary.'''
def reconstruct_path(cameFrom: dict, current: Tuple[int], shape: Tuple[int]) -> List[np.ndarray]:
    path = []
    while current in cameFrom:
        current_array = np.array(current).reshape(shape)
        path.append(current_array)
        current = cameFrom[current]
    path.reverse()  # Reverse to get the path from start to goal
    return path

## Solve the puzzle using A* with explicit gScore and fScore
__Parameters__:
- start_state (np.ndarray): The starting state of the puzzle.
- goal_state (np.ndarray): The goal state of the puzzle.
- heuristic_func (Callable): A function to calculate the heuristic cost.

__Returns__:
- List[np.ndarray]: The sequence of states leading to the solution or None if no solution.

 **Previous attempts**
-  `tentative_gScore = gScore[current_state_tuple] + 1  # Assuming uniform cost of 1 for moves `
- `tentative_gScore = gScore[current_state_tuple] + 0.01  # Assuming uniform cost of 1 for moves`
- `fScore[neighbor_tuple] = tentative_gScore + 1*heuristic_func(neighbor, goal_state)`

In [251]:
#f(n) = g(n) + h(n)
#g is the actual cost
#h is the estimated cost (heuristic)

def a_star_with_scores(start_state: np.ndarray, goal_state: np.ndarray, heuristic_func: Callable[[np.ndarray, np.ndarray], float]) -> List[np.ndarray]:
    
    # Priority queue (open set)
    open_set = []
    
    # Convert states to tuples for immutability
    start_state_tuple = tuple(start_state.flatten())
    goal_state_tuple = tuple(goal_state.flatten())
    
    # Initialize gScore and fScore
    gScore = {start_state_tuple: 0}  # Cost from start to this node
    fScore = {start_state_tuple: heuristic_func(start_state, goal_state)}  # Estimated total cost
    
    # Push the start state into the open set with its heuristic value
    heapq.heappush(open_set, (fScore[start_state_tuple], start_state_tuple))
    
    # Dictionary to reconstruct the path
    cameFrom = {}

    while open_set:
        # Pop the node with the lowest fScore
        _, current_state_tuple = heapq.heappop(open_set)
        current_state = np.array(current_state_tuple).reshape(start_state.shape)

        # Goal test
        if np.array_equal(current_state, goal_state):
            return reconstruct_path(cameFrom, current_state_tuple, start_state.shape)

        # Generate successors (neighbors)
        for action in available_actions(current_state):  # Define available_actions
            neighbor = do_action(current_state, action)  # Define do_action
            neighbor_tuple = tuple(neighbor.flatten())

            # Tentative gScore for the neighbor
            tentative_gScore = gScore[current_state_tuple] + 0.01*heuristic_func(neighbor, goal_state)


            # If this path is better than any previous path to the neighbor
            if tentative_gScore < gScore.get(neighbor_tuple, float('inf')):
                # Record this path as the best so far
                cameFrom[neighbor_tuple] = current_state_tuple
                gScore[neighbor_tuple] = tentative_gScore
                fScore[neighbor_tuple] = tentative_gScore + 1.5*heuristic_func(neighbor, goal_state)

                # Add neighbor to the open set if not already there
                if neighbor_tuple not in [item[1] for item in open_set]:
                    heapq.heappush(open_set, (fScore[neighbor_tuple], neighbor_tuple))

    return None  # Return None if no solution is found


## Initalization
- A solution is always found in fewer than 1000 steps, so 1000 randomized steps are sufficient.

**Previous attempts**
- ` RANDOMIZE_STEPS = 100_000 ` 
- `RANDOMIZE_STEPS = 100 `

In [252]:
Goal_State = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 1000 
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
np.random.seed(42)  # For reproducibility

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))


print("Initial State:")
print(state)

Randomizing: 100%|██████████| 1000/1000 [00:00<00:00, 90155.49it/s]

Initial State:
[[ 2 31 15 18  5  6]
 [33  8 10 23 21  0]
 [27 34  1  4 28 16]
 [26  7 13  3 24 12]
 [ 9 29 32 20 35 11]
 [22 25 30 19 17 14]]


In [253]:

solution_path = a_star_with_scores(state, Goal_State, manhattan_distance)
print("\nSolution Steps:")
print(f"Solution found in {len(solution_path)} moves.")
# if solution_path:
#     print("\nSolution found!")
#     for step, state in enumerate(solution_path):
#         print(f"Step {step}:")
#         print(state)
# else:
#     print("\nNo solution found.")

KeyboardInterrupt: 